# Search Tree Performance Evaluation

In [ ]:
import py4dgeo
import numpy as np
from scipy.spatial import KDTree as ScipyKDTree
from sklearn.neighbors import KDTree as SklearnKDTree
import laspy
from time import perf_counter
import matplotlib.pyplot as plt

The measurement is carried out by decorating functions with the following decorator which will measure the execution time and return it as a tuple with the original return value:

In [ ]:
def measure(f):
    """ A decorator that measures execution time and returns it as part of a tuple """
    def _decorated(*args, **kwargs):
        start = perf_counter()
        ret = f(*args, **kwargs)
        return perf_counter() - start, ret
    return _decorated

To account for noise, small execution times etc., we repeat the function execution a number of times and take the minimum across these runs:

In [ ]:
def minimum_across_runs(n, func, *args):
    measurements = []
    for _ in range(n):
        t, result = func(*args)
        measurements.append(t)
    return min(measurements), result

Our first test is a point cloud of randomly distributed points in the unit cube:

In [ ]:
def create_random_data(n):
    """ Create n samples within the unitcube """
    rng = np.random.default_rng()
    return rng.uniform([0, 0, 0], [1, 1, 1], size=(n, 3)).astype('f')

## Point Cloud Library (PCL)

PCL provides a module `pcl::search` that contains several implementations of a unified search interface: KDTree, OCTree, Bruteforce. The KDTree implementation uses the library FLANN. A general problem of PCL, that it uses a padded data structure for a 3D point to be SSE-friendly (allocating `float[4]`). This hinders seemless interoperability with `numpy.array` as the memory can only be shared if the input data uses that same layout. Currently, the constructor of `py4dgeo.PCLPointCloud` makes a copy of the point cloud.

In [ ]:
@measure
def build_pcl_kdtree(data):
    """ Build PCL KDTree data structure. """
    pc = py4dgeo.PCLPointCloud(data)
    pc.build_tree(py4dgeo.SearchStrategy.kdtree)
    return pc

In [ ]:
@measure
def build_pcl_bruteforce(data):
    """ Build PCL Bruteforce data structure. """
    pc = py4dgeo.PCLPointCloud(data)
    pc.build_tree(py4dgeo.SearchStrategy.bruteforce)
    return pc

In [ ]:
@measure
def build_pcl_octree(data):
    """ Build PCL OCTree data structure. """
    pc = py4dgeo.PCLPointCloud(data)
    pc.build_tree(py4dgeo.SearchStrategy.octree)
    return pc

In [ ]:
@measure
def radius_mine(tree, point, radius):
    """ Invocation of radius search for PCL trees """
    return tree.radius_search(point, radius)

## NanoFLANN

NanoFLANN is a fork of the original FLANN library. Among the reasons to fork, there are several which are beneficial to our use case:

* Single header with permissive license -> Just copy into your project
* Performance gains by removing abstractions
* No approximate searchs (we do not need them?)
* No rigid assumptions on data layout -> Easy to integrate copyless

NanoFLANN is also the search tree library used by PDAL.

In [ ]:
def build_nanoflann(param=10):
    @measure
    def _build_nanoflann(data):
        pc = py4dgeo.NFPointCloud2(data)
        pc.build_tree(param)
        return pc
    return _build_nanoflann

## CloudCompare Core library

I originally promised a reference implementation with [CloudCompare's core library](https://github.com/CloudCompare/CCCoreLib). However, while looking at the library, I turned away for several reasons:

* There is no documentation.
* There is not a single test.
* The PointCloud data structure is quite abstraction-heavy and does not allow copy-less integration with `numpy`.

## Python reference implementations

We use the following two reference implementations: `scipy` and `scikit-learn`. `scipy` has a custom C++ implementation of a KDTree, `scikit-learn` uses Cython. I suspect the latter to be optimized for high dimensional problems.

In [ ]:
def build_scipy(param=10):
    @measure
    def _build_scipy(data):
        """ Build Scipy KDTree data structure """
        return ScipyKDTree(data, leafsize=param)
    return _build_scipy

In [ ]:
@measure
def radius_scipy(tree, point, radius):
    """ Invocation of Scipy radius search """
    return tree.query_ball_point(point, radius)

In [ ]:
@measure
def build_sklearn(data):
    """ Build Sklearn KDTree data structure """
    return SklearnKDTree(data, leaf_size=10)

In [ ]:
@measure
def radius_sklearn(tree, point, radius):
    """ Invocation of Sklearn radius search """
    return tree.query_radius(np.expand_dims(point, axis=0), radius)

## General comparison

The general methodology for the comparison is the following: For an increasing number of points `n` in the point cloud, measure the build time and the query execution time for a radius search individually. We report the KDTree build time as seconds per point in the point cloud. For the radius query, we construct the search radius such that the number of points in the return set stays constant for varying `n` (of course this only holds in a stochastical sense). Consequently, we report the absolute execution time of the query. As the libraries do not provide an interface for performing multiple queries, benchmarking a single query seems okay.

In [ ]:
def compare(impls, max_n=10, radius_scale=2.0):
    nsamples = [2**i * 1000 for i in range(max_n)]
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    for name, build_func, radius_func in impls:
        build_times = []
        query_times = []
        for n in nsamples:
            data = create_random_data(n)
            build_time, cloud = minimum_across_runs(10, build_func, data)
            build_times.append(build_time / n)
            query_time, result = minimum_across_runs(10, radius_func, cloud, np.array([0.5, 0.5, 0.5]), radius_scale * n ** (-(1/3)))
            query_times.append(query_time)
        axs[0].plot(nsamples, build_times, label=name)
        axs[1].plot(nsamples, query_times, label=name)
    axs[0].set_xscale("log")
    axs[1].set_xscale("log")
    axs[0].set_xlabel("Point Cloud size")
    axs[0].set_ylabel("Time/Point [s]")
    axs[1].set_xlabel("Point Cloud size")
    axs[1].set_ylabel("Query Time [s]")
    axs[0].set_title("KDTree build times/point")
    axs[1].set_title("KDTree query time (appr. constant return size)")
    axs[0].legend()
    axs[1].legend()

This is the comparison of the "off-the-shelf" versions of the libraries, where we can clearly see that `SKLearn` is quite a bit slower both for building a querying. This might of course be related to it being optimized for high-dimensionality applications.

In [ ]:
compare([
    ("PCL KDTree", build_pcl_kdtree, radius_mine),
    ("SciPy KDTree", build_scipy(), radius_scipy),
    ("NanoFLANN", build_nanoflann(), radius_mine),
    ("SKLearn KDTree", build_sklearn, radius_sklearn),
])

Looking at the other PCL variants (which were 0 overhead to implement due to the unified interface), we can clearly see that using a KDTree is absolutely mandatory, but we knew that already:

In [ ]:
compare([
    ("PCL KDTree", build_pcl_kdtree, radius_mine),
    ("PCL Bruteforce", build_pcl_bruteforce, radius_mine),
    ("PCL OCtree", build_pcl_octree, radius_mine),
], max_n=8)

Returning to the top contenders, we can see that the performance result actually changes with the search radius (and with it the number of returned points):

In [ ]:
compare([
    ("PCL KDTree", build_pcl_kdtree, radius_mine),
    ("SciPy KDTree", build_scipy(), radius_scipy),
    ("NanoFLANN", build_nanoflann(), radius_mine),
], radius_scale=4.0)

Actually, at some point SciPy has the fastest off-the-shelf implementation:

In [ ]:
compare([
    ("PCL KDTree", build_pcl_kdtree, radius_mine),
    ("SciPy KDTree", build_scipy(), radius_scipy),
    ("NanoFLANN", build_nanoflann(), radius_mine),
], radius_scale=8.0)

## Parameter Tuning

It is worth taking a close look at the parameter tuning for these trees. The most prominent tuning parameter that balances build time vs. query time is the cutoff at which the implementation switches over to a bruteforce algorithm. The implications can be seen e.g. for NanoFLANN:

In [ ]:
compare([
    ("NanoFLANN 5", build_nanoflann(5), radius_mine),
    ("NanoFLANN 10", build_nanoflann(10), radius_mine),
    ("NanoFLANN 20", build_nanoflann(20), radius_mine),
    ("NanoFLANN 50", build_nanoflann(50), radius_mine),
    ("NanoFLANN 100", build_nanoflann(100), radius_mine),
])

A similar study can be done with SciPy, where we see that the qualitative behaviour is similar, but the absolute build times of NanoFLANN are faster in general. This might be related to additional trade-off decisions introduced by the `balanced_tree` and `compact_nodes` parameters of `ScipyKDTree`.

In [ ]:
compare([
    ("SciPy 5", build_scipy(5), radius_scipy),
    ("SciPy 10", build_scipy(10), radius_scipy),
    ("SciPy 20", build_scipy(20), radius_scipy),
    ("SciPy 50", build_scipy(50), radius_scipy),
    ("SciPy 100", build_scipy(100), radius_scipy),
])

The above plots should clearly demonstrate that a finetuning of the cutoff parameter is necessary to achieve optimal performance. In `py4dgeo`, the key factors for the trade-off decisions would be:

* Ratio core-points/points in epoch
* Maximum radius for radius search (or rather # of expected points in radius)

The fact that PCL does not expose the cutoff parameter to users is a clear disadvantage.

## Comparison on LAS data set

The extensive above comparison with points distributed across the unit cube might introduce biases based on the structure of the test cloud. We will therefore verify our findings with a Lidar data set.

In [ ]:
data = np.genfromtxt('/home/jovyan/shared/uls_thingstaette.xyz', delimiter=' ', dtype=np.float32)

In [ ]:
def print_comparison(impls, radius=1.0, runs=10):
    def print_times(name, build_func, radius_func):
        build_time, cloud = minimum_across_runs(runs, build_func, data)
        query_time, result = minimum_across_runs(runs, radius_func, cloud, data[10000], radius)
        print(f"{name} - Build time: {build_time} - Query time {query_time}")
        del cloud
        del result
    for n, bf, rf in impls:
        print_times(n, bf, rf)

In [ ]:
print_comparison([
    ("PCL KDTree", build_pcl_kdtree, radius_mine),
    ("SciPy KDTree", build_scipy(), radius_scipy),
    ("NanoFLANN", build_nanoflann(), radius_mine),
])

The results do not seem to vary too much from what we have seen before, but again we see that SciPy seems to work better for larger return sets:

In [ ]:
print_comparison([
    ("PCL KDTree", build_pcl_kdtree, radius_mine),
    ("SciPy KDTree", build_scipy(), radius_scipy),
    ("NanoFLANN", build_nanoflann(), radius_mine),
], radius=10.0)

Finally, we double-check that our experiments are qualitatively reproduced for a very large dataset of ~180M points.

In [ ]:
rawdata = laspy.read("/home/jovyan/nq071-persistent/ahk_2017_large.laz")
data = np.stack((rawdata.x.array, rawdata.y.array, rawdata.z.array), axis=1)

Unfortunately, the `NanoFLANN` version below is still making the kernel die. I am investigating this.

In [ ]:
print_comparison([
    ("PCL KDTree", build_pcl_kdtree, radius_mine),
    ("SciPy KDTree", build_scipy(), radius_scipy),
#     ("NanoFLANN", build_nanoflann(), radius_mine),
])

## Decision

I am leaning towards a NanoFLANN implementation. A summary of reasons:

* Single Header with BSD License -> Copy into project, no installation etc.
* Flexible handling of input: We can directly operate on `numpy.array` without ugly tricks
* Very competitive performance in above comparisons
* PDAL also uses it.